# Attention is all you need!!

1. `Seq2Seq Modelling and Attention` - A Seq2Seq model takes an input a sequence, and outputs another (like, words, letters or even time series). For e.g., an input of a sentence in German and output sentence in English. A natural choice for such tasks(Neural Machine Translation) is as LSTMs. But they come with their own disadvantages. So the paper [Attention is all you need](https://arxiv.org/abs/1706.03762) describes Transformers. But first, Seq2Seq. Seq2Seq model consists of an Encoder and a Decoder. Encoder maps the features to a higher dimension and Decoder maps this output from Encoder to the lower dimension, maybe equal or lower to the input dimension. Let's add another detail to this - **Attention** . Attention mechanism looks at every input sequence at each iteration and decides which other parts of the sequence are important and need to be paid 'attention'.
> For example in machine translation, Attention mechanism takes 2 sentences of different languages, one as row, other as column and then it makes matches, identifying relevant context. 
If done with the same sentence, this is called 'Self-Attention'

  This helps the neural network to identify the 'meaning', identify the noise and relevant parts. This makes it quite different from memory dependent RNNs or proximity based CNNs. Feed forward networks treat features as independent (gender, siblings); convolutional networks focus on relative location and proximity; RNNs and LSTMs have memory limitations and tend to read in one direction. In contrast to these, attention and the transformer can grab context about a word from distant parts of a sentence, both earlier and later than the word appears, in order to encode information to help us understand the word and its role in the system called a sentence. Now what the hell is a Transformer?

2. `Transformer` - 



In [1]:
import tensorflow as tf
print(tf.__version__)

2.5.0
